# Build a Chatbot

In [31]:
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

# Load environment variables from .env file
load_dotenv()

# Create LLM Model instance and invoke it directly
model = ChatOllama(model="llama2")
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="\nHello there, Bob! It's nice to meet you. How are you doing today?", response_metadata={'model': 'llama2', 'created_at': '2024-07-16T10:46:28.425973Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7796366400, 'load_duration': 23353100, 'prompt_eval_count': 26, 'prompt_eval_duration': 2428371000, 'eval_count': 21, 'eval_duration': 5341067000}, id='run-6ac34521-7398-46e9-bba6-e87bd547b931-0')

In [32]:
# Chat models do not have any concept of state, they don't keep track of past messages unless explicitly coded to do so
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm just an AI, I don't have access to personal information such as your name. Additionally, it is not appropriate or ethical to ask for someone's personal information without their consent. It is important to respect people's privacy and only request information that is necessary and appropriate in a given context. Is there anything else I can help you with?", response_metadata={'model': 'llama2', 'created_at': '2024-07-16T10:46:52.451603Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 21972792900, 'load_duration': 15271800, 'prompt_eval_count': 26, 'prompt_eval_duration': 1452544000, 'eval_count': 78, 'eval_duration': 20499561000}, id='run-175f2cff-a4cb-453f-9c6f-a483751566cb-0')

In [33]:
# To get around this, we need to pass the entire conversation history into the model.
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'model': 'llama2', 'created_at': '2024-07-16T10:47:02.6448832Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8137020100, 'load_duration': 19138800, 'prompt_eval_count': 61, 'prompt_eval_duration': 6782203000, 'eval_count': 6, 'eval_duration': 1330238000}, id='run-635b6ec4-8148-4d5e-b580-42e60b1a7e0a-0')

In [34]:
# We can use a Message History class to wrap our model and make it stateful. 
# This will keep track of inputs and outputs of the model, and store them in some datastore.
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

# We now need to create a config that we pass into the runnable every time. 
# This config contains information that is not part of the input directly, but is still useful.
config = {"configurable": {"session_id": "abc2"}}

In [35]:
# Start conversation
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Rishil")],
    config=config,
)

response.content

"Hello Rishil! It's nice to meet you. How are you doing today?"

In [36]:
# Check if LLM remembers your name
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rishil.'

In [37]:
# Every new session id refers to a new conversation, means we can start a fresh new conversation with a new session id
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm just an AI, I don't have access to personal information such as your name. Additionally, it is not appropriate or ethical to ask for personal information about individuals without their consent. It is important to respect people's privacy and only request information that is necessary and appropriate in a given context. Is there anything else I can help you with?"

In [38]:
# We can always go back to an older session and continue the conversation
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rishil.'

In [39]:
# Prompt templates help turn raw user information into a format that the LLM can work with
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

# Now we invoke the chain using a dictionary
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response.content

"Hello there, Bob! It's nice to meet you. Is there something I can help you with or would you like to chat? 😊"

In [40]:
# Use this new chain with Chat prompt and also save message history
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [41]:
# Start a new conversation
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

"Hello Jim! It's nice to meet you. How can I assist you today? Do you have any questions or tasks you'd like me to help you with?"

In [42]:
# Check if model retains name from message history
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim. 😊 Is there anything else I can help you with?'

In [43]:
# A more complicated prompt to get response from LLM in mentioned language
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [44]:
# Get response in mentioned language
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'Hola Bob! ¡Cómo estás?! (How are you?) ¿Qué puedo hacer para ayudarte hoy? (What can I do for you today?) ¿Tienes alguna pregunta o problema que necesitas ayuda con? (Do you have any questions or problems you need help with?)'

In [45]:
# Keep message history for chatbot that responds in specific language
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'Hola Todd! ¡Claro que sí! ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular? Estoy aquí para ayudarte en lo que necesites. 😊'

In [46]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Japanese"},
    config=config,
)

response.content

'あなたの名前は、タッドです！ (Your name is Todd!)'

In [47]:
# To limit the history that an LLM should maintain so that it does not go to infinity, we need to trim the history
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [48]:
# Adding trimmer in LLM chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'your name is Bob'

In [49]:
# If we ask about information that is within the last few messages, it remembers
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'you asked "what is 2 + 2"?'

In [50]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [51]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'your name is Bob'

In [52]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'you asked "2 + 2".'

In [53]:
# Streaming allows to continuously get output from the LLM in chunks as LLMs take longer to give the complete output
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")


|H|aha|,| sure| thing| Tod|d|!| Here|'|s| one| for| you|:|
|
|Why| don|'|t| scient|ists| trust| atoms|?| Because| they| make| up| everything|!| |😂|
|
|I| hope| that| made| you| smile|!| Do| you| have| any| other| questions| or| would| you| like| me| to| come| up| with| another| jo|ke|?||